In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time


In [2]:

BASE_URL = 'https://www.imdb.com'
def clean_year(year:str) -> int:
    """Clean up a year string."""
    cleaned_year = year.replace('(','').replace(')','')
    try:
        if '–' in cleaned_year:
            years = cleaned_year.split('–')
            return [int(year) for year in years]
        else:
            return int(year.replace('(','').replace(')',''))
    except Exception as e:
        if 'TV Movie' in year or 'Video' in year:
          year = year.split(' ')[0].replace('(','').replace(')','')
        elif any(suffix in year for suffix in ('(I)', '(II)', '(III)','(VII)','VI')):
          year = year.split(' ')[1].replace('(','').replace(')','')
        else:
          return None
 
        return year



def extract_metascore(metascore_tag):
    """Extract metascore from a BeautifulSoup tag."""
    try:
        result = metascore_tag.text.strip()
        return result
    except:
        return None


def extract_votes(votes_gross_tags):
    """Extract votes from a list of BeautifulSoup tags."""
    try:
        votes = votes_gross_tags[0].text
        return votes
    except:
      return None



def extract_gross(votes_gross_tags):
    """Extract gross from a list of BeautifulSoup tags."""
    try:
        if len(votes_gross_tags) !=2:
            gross = None
        else:
            gross = votes_gross_tags[1].text
        return gross
    except:
        return None

def extract_certificate(certificate):
  try:
    rating_cert = certificate.text.replace('-','')
    return rating_cert
  except:
    return None


def extract_runtime(runtime):
  try:
    runtime = runtime.text.split(' ')[0]
    return runtime
  except:
    return None


def extract_ratings(rating):
  try:
    rating = rating.text
    return rating
  except:
    return None

In [3]:

def get_all_data():
    """Scrape data from multiple pages and return as a pandas DataFrame."""
    all_data = []
    for pagenumber in range(0,30):
        url = f'https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page={pagenumber}'
        try:
            response = requests.get(url)
            response.raise_for_status() # Raise an exception if status code is not 200
            print(f'The request to {url} was successful')
            soup = BeautifulSoup(response.content, 'html.parser')
            all_movies = soup.find_all('div',{'class':'lister-item mode-detail'})
            for movie in all_movies:
                data = {}
                data['Title'] = movie.find('h3').find('a').text
                data['Movie Url'] = BASE_URL + movie.find('h3').find('a')['href']
                data['Year'] = clean_year(movie.find('span',{'class':'lister-item-year text-muted unbold'}).text)
                data['Certificate'] = extract_certificate(movie.find('span',{'class','certificate'}))
                data['Genre'] = movie.find('span',{'class','genre'}).text.strip().split(',')
                data['Runtime'] = extract_runtime(movie.find('span',{'class':'runtime'}))
                data['Metascore'] = extract_metascore(movie.find('span',{'class':'metascore'}))
                data['Rating'] = extract_ratings(movie.find('span',{'class','ipl-rating-star__rating'}))
                data['Description'] = movie.find('p',{'class',''}).text.strip()
                data['Director'] = movie.find_all('p',{'class':'text-muted text-small'})[1].find('a').text
                data['Stars'] = [star.text for star in movie.find_all('p',{'class':'text-muted text-small'})[1].find_all('a')[1:]]
                data['Votes'] = extract_votes(movie.find_all('span',{'name':'nv'}))
                data['Gross'] = extract_gross(movie.find_all('span',{'name':'nv'}))
                data['Movie Poster'] = movie.find('div', {'class':'lister-item-image ribbonize'}).find('img')['loadlate']
                all_data.append(data)
        except Exception as e:
            print(f'The requests to {url} failed. {e}')
    return pd.DataFrame(all_data)



In [4]:
movie_dataframe = get_all_data()
movie_dataframe.to_csv('imdb_movies.csv', sep='\t', encoding='utf-8')
movie_dataframe.to_excel('idmb_movies.xlsx', index=False)
movie_dataframe.to_json('idmb.json',orient='records',indent=4)

The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=0 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=1 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=2 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=3 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=4 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=5 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=6 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=&mode=detail&page=7 was successful
The request to https://www.imdb.com/list/ls074451163/?sort=list_order,asc&st_dt=

In [7]:
movie_dataframe.head(10)['Movie Poster'][0]

'https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY209_CR3,0,140,209_AL_.jpg'